# PageRank
This notebook is evaluating the speed of the implemented pagerank vs Networkx solution

In [1]:
import logging
import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg 
import itertools
#from scipy.sparse.linalg import spsolve
#import networkx as nx
#import numpy as np;
#example 1
def create_csr(Z):
    """ Creates a csr presentation from 2darray presentation and 
        calculates the pagerank
    Args:
        G: input graph in the form of a 2d array, such as [[2,0], [1,2], [2,1]]
    Returns:
        Pagerank Scores for the nodes
    
    each row of the array is an edge of the graph [[a,b], [c,d]], a and b are the node numbers. 

    """   
    rows = Z[:,0];
    cols = Z[:,1];
    n = max(max(rows), max(cols))+1;
    G=sprs.csr_matrix((sp.ones(rows.shape),(rows,cols)), shape=(n,n));
    return G

def pagerank_sparse(G, p=0.85, personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph
    
    Args:
        G: a csr graph.
        p: damping factor
        personlize: if not None, should be an array with the size of the nodes
                    containing probability distributions. It will be normalized automatically
        reverse: If true, returns the reversed-pagerank 
        
    Returns:
        Pagerank Scores for the nodes
     
    """
    logger = logging.getLogger(__name__);
    logger.info('started')

    if not reverse:
        G=G.T;

    n,n=G.shape
    c=sp.asarray(G.sum(axis=0)).reshape(-1)
    r=sp.asarray(G.sum(axis=1)).reshape(-1)

    k=c.nonzero()[0]

    D=sprs.csr_matrix((1/c[k],(k,k)),shape=(n,n))

    if personalize is None:
        e=sp.ones((n,1))
    else:
        e = personalize/sum(personalize);
        
    I=sprs.eye(n)
    X1 = sprs.linalg.spsolve((I - p*G.dot(D)), e);

    X1=X1/sum(X1)
    logger.info('finished')
    return X1
def pagerank_sparse_power(G, p=0.85, max_iter = 100, personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph
    
    Args:
        G: a csr graph.
        p: damping factor
        max_iter: maximum number of iterations
        personlize: if not None, should be an array with the size of the nodes
                    containing probability distributions. It will be normalized automatically
        reverse: If true, returns the reversed-pagerank 
        
    Returns:
        Pagerank Scores for the nodes
     
    """
    logger = logging.getLogger(__name__);
    logger.info('started')
    
    if not reverse: 
        G=G.T;

    n,n=G.shape
    c=sp.asarray(G.sum(axis=0)).reshape(-1)
    r=sp.asarray(G.sum(axis=1)).reshape(-1)

    k=c.nonzero()[0]

    D=sprs.csr_matrix((1/c[k],(k,k)),shape=(n,n))

    if personalize is None:
        e=sp.ones((n,1))
    else:
        e = personalize/sum(personalize);
        
    z = (((1-p)*(c!=0) + (c==0))/n)[sp.newaxis,:]
    G = p*G.dot(D)
    x = e/n
    oldx = sp.zeros((n,1));
    
    iteration = 0
    start = time.time()    
    while sp.linalg.norm(x-oldx) > 0.01:
        oldx = x
        x = G.dot(x) + e.dot(z.dot(x))
        iteration += 1
        if iteration >= max_iter:
            break;
    #print "here"
    #print time.time()-start            
    x = x/sum(x)
    
    logger.info('finished')
    return x.reshape(-1)
    
def pagerank_netx(Z, personalization=None):
    G=nx.DiGraph()
    G.add_edges_from(Z)
    PR=nx.pagerank(G, personalization=personalization);
    #PR=nx.pagerank_numpy(G);
    #PR=nx.pagerank_scipy(G);
    P=sp.array([PR[i] for i in range(len(PR))])
    return P
    


In [26]:
import time
# http://www.cs.princeton.edu/~chazelle/courses/BIB/pagerank.htm

#example 1
Z=sp.array([[1,2],[1,3],[2,3],[3,1],[4,3]]);

# example 2
#Z=sp.array([[1,2],[2,1],[1,3],[3,1],[1,4],[4,1],[4,5],[4,6],[4,7],[4,8]])


# Example 11
# Z=sp.array([[1,2],[2,1],[2,3],[3,1],[3,4],[4,1]])
Z=Z-1

#Z=scipy.sparse.rand(10, 10, density=0.3, format='csr');

G=create_csr(Z)
per = sp.zeros((G.shape[0],1))
               
per[0]=0.5
per[1]=0.25
per[2]=0.25

start = time.time()
print pagerank_sparse(G, personalize=per)
print time.time()-start

start = time.time()
print pagerank_sparse_power(G, personalize=per)
print time.time()-start

per = dict(enumerate(sp.squeeze(per).tolist()))
print per
start = time.time()
print pagerank_netx(Z, personalization=per)
print time.time()-start


  (1, 0)	1.0
  (2, 0)	1.0
  (2, 1)	1.0
  (0, 2)	1.0
  (2, 3)	1.0
[ 0.40390051  0.20915772  0.38694178  0.        ]
0.00373101234436
[ 0.40529564  0.19358685  0.40111751  0.        ]
0.00258302688599
{0: 0.5, 1: 0.25, 2: 0.25, 3: 0.0}
[ 0.40389993  0.20915739  0.38694267  0.        ]
0.00159811973572


In [3]:
import sys
n=30000
Z= scipy.sparse.rand(n, n, density=0.01, format='dok')
for i,j in Z.keys():
         Z[i, j] = 1
print "started"        
sys.stdout.flush()
start = time.time()
pagerank_sparse(Z.asformat('csr'))
print time.time()-start

start = time.time()
pagerank_sparse_power(Z.asformat('csr'))
print time.time()-start

start = time.time()
pagerank_netx(Z.keys())
print time.time()-start



started


NameError: name 'time' is not defined

# Union

In [3]:
import itertools
import scipy as sp
def _e2i(*iters):
    elist=[];
    edict=dict();
    last=0;    
    for wid in itertools.chain(*iters):
        if wid not in edict:
            edict[wid]=last;
            elist.append(wid);
            last +=1; 
    return elist, edict;

def _unify_ids_scores(*id_sc_tuple):
    uids, id2in =_e2i(*(ids for ids, _ in id_sc_tuple));
    
    uscs=tuple();            
    for ids,scs in id_sc_tuple:
        scs_u=sp.zeros(len(id2in))
        scs_u[[id2in[wid] for wid in ids]] = scs;            
        uscs += (scs_u,)                
    return uids, uscs, id2in       


def _mergelinks(*ids_scs_links):
    uids, uscs, uid2in  = _unify_ids_scores(*((ids, scs) for ids,scs,_ in ids_scs_links));
    mergedlinks=set();
    for ids, _,links in ids_scs_links:
        for u,v in links:
            mergedlinks.add((uid2in[ids[u]],uid2in[ids[v]]))
    mergedlinks = sp.array([[u,v] for u,v in mergedlinks])            
    return uids, uscs, mergedlinks

In [13]:
from collections import defaultdict
ids1=[2,3,5,12,1]
sc1=[22,33,44,55,66]

em1=defaultdict(int, zip(ids1, sc1))


ids2=[1,2,7]
sc2=[99,100,101]
em2=defaultdict(int, zip(ids2, sc2))


uds, (usc1, usc2), _ = _unify_ids_scores((ids1,sc1),(ids2,sc2))
print uds
print usc1
print usc2

uds=list(set(em1.keys()).union(em2.keys()))
usc1=[em1[wid] for wid in uds]
usc2=[em2[wid] for wid in uds]

print uds
print usc1
print usc2



[2, 3, 5, 12, 1, 7]
[ 22.  33.  44.  55.  66.   0.]
[ 100.    0.    0.    0.   99.  101.]
[1, 2, 3, 5, 7, 12]
[66, 22, 33, 44, 0, 55]
[99, 100, 0, 0, 101, 0]


In [2]:
ids1=[2,3,5,12,1]
sc1=[22,33,44,55,66]
t1=['a', 'b', 'c', 'd']
links1 = sp.array([[0,2], [4,0], [0,3]])

links1id = set((ids1[u],ids1[v]) for u,v in links1)

ids2=[1,2,7]
sc2=[99,100,101]
t2=['a', 'b', 'c', 'd']
links2 = sp.array([[0,1], [2,0], [2,1]])
links2id = set((ids2[u],ids2[v]) for u,v in links2)



ids3=[5,18, 7]
sc3=[34,43, 123]
t3=['jack', 'joe', 'khose']
links3=sp.array([[0,2], [1,2]])
links3id = set((ids3[u],ids3[v]) for u,v in links3)

print links1id
print links2id
print links3id

print "\n"

uds, (usc1, usc2, usc3), _ = _unify_ids_scores((ids1,sc1),(ids2,sc2),(ids3,sc3))


print uds
print usc1
print usc2
print usc3

print "\n"


uids, (usc1, usc2, usc3), mergedlinks = _mergelinks((ids1,sc1,links1),(ids2,sc2,links2),(ids3,sc3,links3))
print uids
print usc1
print usc2
print usc3
print mergedlinks
linksu = set((uids[u],uids[v]) for u,v in mergedlinks)

print linksu


set([(1, 2), (2, 5), (2, 12)])
set([(1, 2), (7, 2), (7, 1)])
set([(18, 7), (5, 7)])


[2, 3, 5, 12, 1, 7, 18]
[ 22.  33.  44.  55.  66.   0.   0.]
[ 100.    0.    0.    0.   99.  101.    0.]
[   0.    0.   34.    0.    0.  123.   43.]


[2, 3, 5, 12, 1, 7, 18]
[ 22.  33.  44.  55.  66.   0.   0.]
[ 100.    0.    0.    0.   99.  101.    0.]
[   0.    0.   34.    0.    0.  123.   43.]
[[5 4]
 [0 2]
 [5 0]
 [2 5]
 [0 3]
 [6 5]
 [4 0]]
set([(1, 2), (7, 1), (5, 7), (18, 7), (2, 12), (2, 5), (7, 2)])


[2.3, 3.5, 4.0]
